In [97]:
import sagemaker
from sagemaker import image_uris, get_execution_role
import boto3
import numpy as np
from sklearn.metrics import confusion_matrix
import glob
from sagemaker.predictor import Predictor
from sagemaker.serializers import IdentitySerializer
import base64
import json
from sklearn.metrics import classification_report
import time

In [98]:
role = get_execution_role()
role

'arn:aws:iam::968765902470:role/service-role/AmazonSageMaker-ExecutionRole-20240509T201394'

In [99]:
"""
Retrieving the URI for the SageMaker built-in image classification algorithm container.
'region' specifies the AWS region (obtained from the current boto3 session),
and 'framework' specifies the built-in algorithm name ("image-classification").
"""
algorithm_image = image_uris.retrieve(region=boto3.Session().region_name,
                                      framework="image-classification")

In [100]:
# Define the SageMaker model
model = sagemaker.model.Model(
    image_uri=algorithm_image,  # The URI of the Docker image for the SageMaker built-in image classification algorithm
    # model_data="s3://medical--ai-chest-xray/models/image_model/classifier-2024-05-26-07-20-20-003-6efa17fb/output/model.tar.gz",  # S3 URI of the model artifacts
    model_data="s3://medical--ai-chest-xray/models/image_model/classifier-2024-05-27-07-10-10-091-000007ca/output/model.tar.gz",
    role=role                   # The IAM role that SageMaker uses to access AWS resources
)


In [101]:
endpoint_name = "Demo-image-classifier-Pneumonia2"

In [102]:


# You can deploy the model to an endpoint to make predictions
deployment = model.deploy(
    initial_instance_count=1,      # The number of EC2 instances to deploy the model on
    instance_type="ml.m5.xlarge",  # The type of EC2 instance to use for the deployment
    endpoint_name=endpoint_name
)

----------------------------------------------------*

UnexpectedStatusException: Error hosting endpoint Demo-image-classifier-Pneumonia2: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html

In [37]:
predictor = Predictor(endpoint_name)
predictor.serializer = IdentitySerializer("image/jpeg")

In [38]:
file_path = "data/chest_xray/val/*.jpeg"
files = glob.glob(file_path)

y_true = []
y_pred = []


def make_pred():
    for file in files:
        if "normal" in file:
            with open(file, "rb") as f:
                payload = f.read()
                inference = predictor.predict(data=payload).decode("utf-8")
                result = json.loads(inference)
                pred_class = np.argmax(result)
                y_true.append(0)
                y_pred.append(pred_class)
        elif "pneumonia" in file:
            with open(file, "rb") as f:
                payload = f.read()
                inference = predictor.predict(data=payload).decode("utf-8")
                result = json.loads(inference)
                pred_class = np.argmax(result)
                y_true.append(1)
                y_pred.append(pred_class)

make_pred()
print(y_true)
print(y_pred)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]


In [39]:
confusion_matrix(y_true, y_pred)

array([[2, 6],
       [0, 8]])

In [41]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.25      0.40         8
           1       0.57      1.00      0.73         8

    accuracy                           0.62        16
   macro avg       0.79      0.62      0.56        16
weighted avg       0.79      0.62      0.56        16

